<font color="#CC3D3D"><p>
# [Competition] Building a `LightGBM` Model with `Pipeline+Optuna`

In [1]:
LGBM_VERSION = 4.0   # submission 화일명에 사용

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import TransformedTargetRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from category_encoders import CatBoostEncoder
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV
from optuna.integration.lightgbm import LightGBMTunerCV
from scipy.sparse import csr_matrix
import re

In [3]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [4]:
X_train['직무태그'] = X_train['직무태그'].str.replace(' ','')
X_train['직무태그'] = X_train['직무태그'].fillna('없음')
X_train['직무태그'] = X_train['직무태그'].str.lower()
X_test['직무태그'] = X_test['직무태그'].str.replace(' ','')
X_test['직무태그'] = X_test['직무태그'].fillna('없음')
X_test['직무태그'] = X_test['직무태그'].str.lower()

In [5]:
## 어학시험 결측치 처리
X_train['어학시험'] = X_train['어학시험'].fillna('없음')
X_train = X_train.replace({'어학시험' :' '},'없음') 

X_test['어학시험'] = X_test['어학시험'].fillna('없음')
X_test = X_test.replace({'어학시험' :' '},'없음') 

In [6]:
for i , j in  zip(X_train['대학전공'].str.contains('호텔'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'호텔',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('관광'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'관광',inplace=True)   
        
for i , j in  zip(X_train['대학전공'].str.contains('체육'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'체육',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('무역'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'무역',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('국제통상'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'국제통상',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('통계'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'통계',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('조리'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'조리',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('경영'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'경영',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('경제'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'경제',inplace=True)

for i , j in  zip(X_train['대학전공'].str.contains('디자인'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'디자인',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('경제'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'경제',inplace=True)

for i , j in  zip(X_train['대학전공'].str.contains('컴퓨터'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'컴퓨터',inplace=True)

for i , j in  zip(X_train['대학전공'].str.contains('교육'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'교육',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('스포츠'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'체육',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('영어'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'영어영문',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('중국어'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'중어중문',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('중국'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'중국학과',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('광고'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'광고',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('미술'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'미술',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('정치'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'정치',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('전기'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'전기',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('국문'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'국문',inplace=True)

for i , j in  zip(X_train['대학전공'].str.contains('의류'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'의류',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('패션'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'의류',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('화학'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'화학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('생명'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'생명',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('물리'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'물리',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('사학'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'사학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('행정'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'행정',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('통상'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'국제통상',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('산업'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'산업공학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('심리'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'심리',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('법'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'법학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('조형'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'조형',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('수학'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'수학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('문헌'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'문헌정보',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('회계'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'회계',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('정보'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'정보통신',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('중어'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'중어중문',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('전자'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'전자과',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('독어'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'독어독문',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('일어'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'일어일문',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('불어'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'불어일문',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('토목'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'토목',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('건축'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'건축',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('전산'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'전산학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('기계'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'기계공학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('복지'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'복지',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('사회'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'사회학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('보험'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'보험',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('금융'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'금융',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('환경'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'환경',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('금속'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'금속공학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('도시'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'도시',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('미디어'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'멀티미디어',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('제어'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'제어계측',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('신소재'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'신소재',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('신문'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'신문방송',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('통신'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'정보통신',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('식품'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'식품영양',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('생물'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'생물학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('유전'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'유전공학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('가정'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'가정',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('소프트'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'컴퓨터',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('지리'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'지리학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('지리'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'지리학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('섬유'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'섬유공학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('의상'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'의상학과',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('보건'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'보건관리',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('신학'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'신학과',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('신학'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'신학과',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('자원'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'자원공학',inplace=True)
                                
for i , j in  zip(X_train['대학전공'].str.contains('국제'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'국제통상',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('공예'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'공예',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('일본'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'일어일문',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('메카'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'메카트로닉스',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('영문'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'영어영문',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('간호'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'간호학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('영양'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'식품영양',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('분자'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'고분자',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('아동'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'아동학',inplace=True)
        
for i , j in  zip(X_train['대학전공'].str.contains('무용'), X_train['대학전공']) :
    if i ==True:
        X_train['대학전공'].replace(j,'무용',inplace=True)

In [7]:
for i , j in  zip(X_test['대학전공'].str.contains('호텔'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'호텔',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('관광'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'관광',inplace=True)   
        
for i , j in  zip(X_test['대학전공'].str.contains('체육'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'체육',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('무역'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'무역',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('국제통상'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'국제통상',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('통계'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'통계',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('조리'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'조리',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('경영'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'경영',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('경제'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'경제',inplace=True)

for i , j in  zip(X_test['대학전공'].str.contains('디자인'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'디자인',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('경제'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'경제',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('컴퓨터'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'컴퓨터',inplace=True)

for i , j in  zip(X_test['대학전공'].str.contains('교육'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'교육',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('스포츠'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'체육',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('영어'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'영어영문',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('중국어'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'중어중문',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('중국'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'중국학과',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('광고'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'광고',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('미술'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'미술',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('정치'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'정치',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('전기'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'전기',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('국문'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'국문',inplace=True)

for i , j in  zip(X_test['대학전공'].str.contains('의류'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'의류',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('패션'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'의류',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('화학'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'화학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('생명'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'생명',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('물리'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'물리',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('사학'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'사학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('행정'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'행정',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('통상'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'국제통상',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('산업'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'산업공학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('심리'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'심리',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('법'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'법학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('조형'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'조형',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('수학'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'수학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('문헌'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'문헌정보',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('회계'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'회계',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('정보'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'정보통신',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('중어'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'중어중문',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('전자'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'전자과',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('독어'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'독어독문',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('일어'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'일어일문',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('불어'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'불어일문',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('토목'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'토목',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('건축'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'건축',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('전산'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'전산학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('기계'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'기계공학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('복지'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'복지',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('사회'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'사회학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('보험'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'보험',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('금융'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'금융',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('환경'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'환경',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('금속'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'금속공학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('도시'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'도시',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('미디어'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'멀티미디어',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('제어'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'제어계측',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('신소재'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'신소재',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('신문'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'신문방송',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('통신'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'정보통신',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('식품'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'식품영양',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('생물'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'생물학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('유전'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'유전공학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('가정'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'가정',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('소프트'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'컴퓨터',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('지리'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'지리학',inplace=True)

for i , j in  zip(X_test['대학전공'].str.contains('섬유'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'섬유공학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('의상'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'의상학과',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('보건'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'보건관리',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('신학'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'신학과',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('신학'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'신학과',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('자원'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'자원공학',inplace=True)
                                
for i , j in  zip(X_test['대학전공'].str.contains('국제'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'국제통상',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('공예'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'공예',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('일본'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'일어일문',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('메카'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'메카트로닉스',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('영문'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'영어영문',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('간호'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'간호학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('영양'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'식품영양',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('분자'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'고분자',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('아동'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'아동학',inplace=True)
        
for i , j in  zip(X_test['대학전공'].str.contains('무용'), X_test['대학전공']) :
    if i ==True:
        X_test['대학전공'].replace(j,'무용',inplace=True)

In [8]:
df=pd.concat([X_train, X_test])

df['근무지역']=df['근무지역'].replace({
                            '서울,서울,서울':'서울',
                            '서울,서울,':'서울',
                            '서울,,':'서울',
                            '서울,,서울':'서울'})                         
df['근무형태']=df['근무형태'].replace(
{'정규직, 계약직':'정규직',
'정규직, 해외취업':'정규직',
'정규직, 계약직, ':'정규직',
'정규직, ':'정규직',
'정규직, 인턴':'정규직',
'정규직, 계약직, 해외취업':'정규직',
'정규직, 계약직, 인턴':'정규직',
'정규직, 계약직, 파견직':'정규직',
'정규직, 계약직, 해외취업, ':'정규직',
'정규직, 계약직, 파견직, ':'정규직',
'정규직, 계약직, 해외취업, 파견직':'정규직',
'계약직, ':'계약직',
'정규직, 계약직, 해외취업, 파견직, ':'정규직',
'정규직, 해외취업, 파견직':'정규직',
'정규직, 계약직, 해외취업, 인턴':'정규직',
'정규직, 파견직':'정규직',
'정규직, 계약직, 인턴, 파견직':'정규직',
'정규직, 해외취업, ':'정규직',
'해외취업':'기타',
'정규직, 계약직, 해외취업, 인턴, 파견직, ':'정규직',
'정규직, 계약직, 인턴, 파견직, ':'정규직',
'정규직, 병역특례':'정규직',
'정규직, 계약직, 인턴, ':'정규직',
'파견직':'기타',
'정규직, 파견직, ':'정규직',
'정규직, 해외취업, 인턴':'정규직',
'인턴':'기타',
'정규직, 계약직, 해외취업, 인턴, 파견직':'정규직',
'정규직, 해외취업, 파견직, ':'정규직',
'계약직, 해외취업':'계약직',
'계약직, 파견직, ':'계약직',
'정규직, 계약직, 해외취업, 인턴, ':'정규직',
'정규직, 해외취업, 병역특례':'정규직',
'계약직, 파견직':'계약직',
'계약직, 인턴':'계약직',
'정규직, 인턴, ':'정규직',
'정규직, 해외취업, 인턴, 파견직':'정규직',
'계약직, 해외취업, 파견직, ':'계약직',
'병역특례':'기타',
'계약직, 인턴, ':'계약직',
'정규직, 해외취업, 인턴, ':'정규직',
'인턴, ':'기타',
'계약직, 해외취업, 인턴':'계약직',
'정규직, 병역특례, 인턴':'정규직',
'정규직, 계약직, 해외취업, 병역특례, 인턴, 파견직, ':'정규직',
'정규직, 병역특례, 파견직':'정규직'})

df['근무형태'] = df['근무형태'].fillna('신입')

df['근무경력'].str.split(" ").str[0]
df['근무경력'].str.split(" ").str[-1]


df['연']=df['근무경력'].str.split(" ").str[0]
df['개월']=df['근무경력'].str.split(" ").str[-1]
df1=pd.concat([df.연,df.개월],axis=1)

df1['연'].loc[df1['연'] == '0개월'] = '0년'
df1['연'].loc[df1['연'] == '1개월'] = '0년'
df1['연'].loc[df1['연'] == '2개월'] = '0년'
df1['연'].loc[df1['연'] == '3개월'] = '0년'
df1['연'].loc[df1['연'] == '4개월'] = '0년'
df1['연'].loc[df1['연'] == '5개월'] = '0년'
df1['연'].loc[df1['연'] == '6개월'] = '0년'
df1['연'].loc[df1['연'] == '7개월'] = '0년'
df1['연'].loc[df1['연'] == '8개월'] = '0년'
df1['연'].loc[df1['연'] == '9개월'] = '0년'
df1['연'].loc[df1['연'] == '10개월'] = '0년'
df1['연'].loc[df1['연'] == '11개월'] = '0년'

df1['A'] = df1['연'].str.rstrip("년")
df1['B'] = df1['개월'].str.rstrip("개월")
df1['A'].unique()

df1['연차를 개월로 환산'] = df1['A'].astype(dtype='int64')*12
df1['B'] = df1['B'].astype(dtype='int64')*1
df1['개월수'] = df1['연차를 개월로 환산']+df1['B']
df['근무경력'] = df1['개월수']
df['근무경력'].value_counts()

df=df.drop(['연'],axis=1)
df=df.drop(['개월'],axis=1)

In [9]:
for i,j in zip(df['근무경력'],df['근무형태']):
    if (i != 0.0 ) and (j =='신입'):
        df['근무형태_new'] = df['근무형태'].replace(j,'정규직',inplace=True)

In [10]:
X_train = df[:16570]
X_test = df[16570:]

In [11]:
numeric_features = ['근무경력','대학성적'] 
categorical_features = ['근무형태','출신대학','대학전공','어학시험','자격증']
binary_features = ['직종','세부직종','근무지역']
pca_features = ['직무태그']

X_train = X_train[numeric_features+categorical_features+binary_features+pca_features]  # 순서 주의!!!
X_test = X_test[numeric_features+categorical_features+binary_features+pca_features]

X_train.shape, X_test.shape

((16570, 11), (11048, 11))

In [12]:
def remove_outlier(X, q=0.05):  
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})), 
        ("scaler", PowerTransformer()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int)),
    ]
)
    
binary_transformer = Pipeline(
    steps=[
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
    ]
)

pca_transformer = Pipeline(
    steps=[
        ("impuer", FunctionTransformer(lambda x: x.fillna('없음'))),      
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
        ("dense", FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
        ("pca", PCA(n_components=200)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("bin1", binary_transformer, binary_features[0]), 
        ("bin2", binary_transformer, binary_features[1]), 
        ("bin3", binary_transformer, binary_features[2]), 
        ("pca1", pca_transformer, pca_features[0]), 
   ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
        ("selector", SelectPercentile(percentile=80)),
    ]
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", LGBMRegressor(random_state=0)),
    ]
)

set_config(display="diagram")  # To view the text pipeline, change to display='text'.
model

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('column',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('outlier',
                                                                                    FunctionTransformer(func=<function remove_outlier at 0x00000196DB062790>,
                                                                                                        kw_args={'q': 0.05})),
                                                                                   ('scaler',
                                                                                    PowerTransformer())]),
                                                                   ['근무경력',
                                                                    '대학성적']),
                                                                  ('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='...
                                                                                    FunctionTransformer(func=<function <lambda> at 0x00000196DB0628B0>)),
                                                                                   ('corpus',
                                                                                    FunctionTransformer(func=<function <lambda> at 0x00000196DB0621F0>)),
                                                                                   ('BoW',
                                                                                    CountVectorizer()),
                                                                                   ('dense',
                                                                                    FunctionTransformer(accept_sparse=True,
                                                                                                        func=<function <lambda> at 0x00000196DB062160>)),
                                                                                   ('pca',
                                                                                    PCA(n_components=200))]),
                                                                   '직무태그')])),
                                 ('selector',
                                  SelectPercentile(percentile=80))])),
                ('classifier', LGBMRegressor(random_state=0))])

In [13]:
sscv = ShuffleSplit(test_size=.3334, n_splits=5, random_state=0) # Public LB 사이즈와 동일하게 평가데이터 사이즈 설정 
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=sscv)

print("Default LGBM CV scores: ", np.sqrt(-1*scores))
print("Default LGBM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

Default LGBM CV scores:  [855.68409459 848.1391502  871.90229717 895.36923861 886.72643389]
Default LGBM CV mean = 871.75 with std = 176.52


In [14]:
%%time

param_distributions = {
    "preprocessor__column__num__imputer__strategy": CategoricalDistribution(["mean"]),
    "preprocessor__column__num__outlier__kw_args": CategoricalDistribution([{'q':0.02},{'q':0.03}]),
    "preprocessor__column__pca1__pca__n_components": IntDistribution(100,500,step=100),  
    "preprocessor__selector__percentile": IntDistribution(70,100,step=10),  
}

optuna_search = OptunaSearchCV(model, 
                               param_distributions, 
                               cv=sscv, scoring='neg_mean_squared_error', 
                               n_trials=20,
                               timeout=60, 
                               study=optuna.create_study(sampler=optuna.samplers.TPESampler(seed=100), direction="maximize"))

optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna_search.fit(X_train, y_train)

print(f"Best params: {optuna_search.best_params_}")
print("Best score: %.2f" % (-1*optuna_search.best_score_)**0.5)

[I 2022-11-24 20:36:10,972] A new study created in memory with name: no-name-bd57187c-5598-45b9-8059-c61b1c090b94


Best params: {'preprocessor__column__num__imputer__strategy': 'mean', 'preprocessor__column__num__outlier__kw_args': {'q': 0.02}, 'preprocessor__column__pca1__pca__n_components': 300, 'preprocessor__selector__percentile': 100}
Best score: 855.52
CPU times: total: 6min 28s
Wall time: 1min 17s


In [15]:
# 최적값으로 파이프라인 재설정
model.set_params(**optuna_search.best_params_)

# 전처리 파이프라인만 수행
X_train = preprocessor.fit_transform(X_train, y_train)
X_test = preprocessor.transform(X_test)

In [17]:
tuner = LightGBMTunerCV(
    params={
        "objective": "regression",   # 지도학습 유형(regression/binary/multiclass)
        "metric": "rmse",
        "verbosity": -1,             # 진행과정 출력안함
        "boosting_type": "gbdt",     # 실행하고자 하는 알고리즘 유형(gdbt/rf/dart/goss)
        "seed": 0,
        "boost_from_average" : False,
        "Approximate method" : "exact",
        "num_iterations": 1000,
        "learning_rate": 0.05,
        "max_depth": 30,
        "feature_fraction": 0.9
    },
    train_set=lgb.Dataset(X_train, y_train), # LGBM 데이터셋으로 변환
    nfold=5,
    num_boost_round=500,                     # boosting 반복횟수
    callbacks=[lgb.early_stopping(100)],     # 모든 정지 라운드 동안 validation 성능이 개선되지 않으면 조기종료
    time_budget=500,                          # 튜닝 허용 최대시간(초단위)
    optuna_seed=0,
    
)

tuner.run()


feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds



feature_fraction, val_score: 842.628008:  14%|######1                                    | 1/7 [00:11<01:09, 11.50s/it]

Early stopping, best iteration is:
[448]	cv_agg's rmse: 842.628 + 4.77818



feature_fraction, val_score: 842.628008:  29%|############2                              | 2/7 [00:28<01:13, 14.66s/it]

...

Early stopping, best iteration is:
[509]	cv_agg's rmse: 842.151 + 4.7492



feature_fraction, val_score: 842.150980: 100%|###########################################| 7/7 [01:35<00:00, 13.65s/it]


Early stopping, best iteration is:
[509]	cv_agg's rmse: 842.151 + 4.7492



num_leaves, val_score: 842.150980:   0%|                                                        | 0/20 [00:00<?, ?it/s][W 2022-11-24 20:46:20,748] Trial 7 failed because of the following error: LightGBMError('Check failed: (num_leaves) <= (131072) at D:\\a\\1\\s\\python-package\\compile\\src\\io\\config_auto.cpp, line 335 .\n')
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\user\anaconda3\lib\site-packages\optuna\integration\_lightgbm_tuner\optimize.py", line 319, in __call__
    cv_results = lgb.cv(self.lgbm_params, train_set, **self.lgbm_kwargs)
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py", line 605, in cv
    cvfolds = _make_n_folds(train_set, folds=folds, nfold=nfold,
  File "C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py", line 361, in _make_n_folds
    full_data = full_data.construct()
  File "C:\Users

LightGBMError: Check failed: (num_leaves) <= (131072) at D:\a\1\s\python-package\compile\src\io\config_auto.cpp, line 335 .


In [ ]:
print(f"\nBest params: {tuner.best_params}")
print(f"\nBest score: {tuner.best_score:.2f}")

In [ ]:
# 최적화된 하이퍼파라미터로 OOF를 수행하여 최종 LGBM 모형 생성
models = cross_validate(LGBMRegressor(**tuner.best_params), # 최적화된 hyperparameter 사용
                        X_train, y_train, 
                        cv=sscv, 
                        scoring='neg_mean_squared_error', 
                        return_estimator=True)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

scores = models['test_score']
print("\nTuned LGBM CV scores: ", np.sqrt(-1*scores))
print("Tuned LGBM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

In [ ]:
# submission 화일 생성
filename = f'lgbm_{LGBM_VERSION}_{np.sqrt(-1*scores.mean()):.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)